<a href="https://colab.research.google.com/github/thuyk39nguyen/AIO/blob/main/Modul6_week1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **2.1.1. Bộ dữ liệu MNIST**

1. Tải về bộ dữ liệu

In [ ]:
# Import libs
import os
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from torchsummary import summary

import matplotlib.pyplot as plt
from PIL import Image

ROOT = './data'
train_data = datasets.MNIST(root = ROOT, train = True, download = True)
test_data = datasets.MNIST(root = ROOT, train = False, download = True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 53.5MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 1.76MB/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 12.5MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 7.32MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



2. Tiền xử lý dữ liệu
- Chia tỉ lệ các tập training: validation = 0.9 : 0.1
- Chuẩn hoá dữ liệu và chuyển sang tensor sử dụng torchvison.transform

In [ ]:
# split training : validation = 0.9 : 0.1
VALID_RATIO = 0.9

n_train_examples = int(len(train_data) * VALID_RATIO)
n_valid_examples = len(train_data) - n_train_examples
train_data, valid_data = data.random_split (train_data, [n_train_examples, n_valid_examples])

# compute mean and std for normalization
mean = train_data.dataset.data.float().mean()/255
std = train_data.dataset.data.float().std()/255

train_transforms = transforms.Compose ([transforms.ToTensor(), transforms.Normalize(mean =[ mean ] , std =[ std ])])
test_transforms = transforms.Compose ([transforms.ToTensor(), transforms.Normalize(mean =[ mean ] , std =[ std ])])

train_data.dataset.transform = train_transforms
valid_data.dataset.transform = test_transforms

# Create dataloader

BATCH_SIZE = 256
train_dataloader = data.DataLoader(train_data, shuffle = True, batch_size = BATCH_SIZE)
valid_dataloader = data.DataLoader(valid_data, batch_size = BATCH_SIZE)

3. Xây dựng mô hình LeNet

In [ ]:
class LeNetClassifier ( nn.Module ) :
  def __init__ ( self , num_classes ) :
    super ().__init__ ()
    self.conv1 = nn.Conv2d (in_channels =1 , out_channels =6 , kernel_size =5 , padding ='same')
    self.avgpool1 = nn.AvgPool2d ( kernel_size =2)
    self.conv2 = nn.Conv2d ( in_channels =6 , out_channels =16 , kernel_size =5)
    self.avgpool2 = nn.AvgPool2d ( kernel_size =2)
    self.flatten = nn.Flatten ()
    self.fc_1 = nn.Linear (16 * 4 * 4 , 120)
    self.fc_2 = nn.Linear (120 , 84)
    self.fc_3 = nn.Linear (84 , num_classes )

  def forward ( self , inputs ) :
    outputs = self.conv1 ( inputs )
    outputs = self.avgpool1 ( outputs )
    outputs = F.relu ( outputs )
    outputs = self.conv2 ( outputs )
    outputs = self.avgpool2 ( outputs )
    outputs = F.relu ( outputs )
    outputs = self.flatten ( outputs )
    outputs = self.fc_1 ( outputs )
    outputs = self.fc_2 ( outputs )
    outputs = self.fc_3 ( outputs )
    return outputs


In [ ]:
class LeNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding='same')
        self.avgpool1 = nn.AvgPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.avgpool2 = nn.AvgPool2d(kernel_size=2)
        self.flatten = nn.Flatten()
        # The issue was with this line, originally defined with an input size of 16 * 4 * 4 = 256
        # We need to adjust the input size to match the actual output size of the convolutional layers.
        # I will replace this with a calculated value to showcase adaptability but you could replace with 400
        # calculate the output size dynamically
        x = torch.randn(1, 1, 28, 28)  # Sample input of shape (batch_size, channels, height, width)
        x = self.conv1(x)
        x = self.avgpool1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.avgpool2(x)
        x = F.relu(x)
        x = self.flatten(x)
        input_size = x.shape[1]

        self.fc_1 = nn.Linear(input_size, 120)  # Adjust the input size here
        self.fc_2 = nn.Linear(120, 84)
        self.fc_3 = nn.Linear(84, num_classes)

    def forward(self, inputs):
        outputs = self.conv1(inputs)
        outputs = self.avgpool1(outputs)
        outputs = F.relu(outputs)
        outputs = self.conv2(outputs)
        outputs = self.avgpool2(outputs)
        outputs = F.relu(outputs)
        outputs = self.flatten(outputs)
        outputs = self.fc_1(outputs)
        outputs = self.fc_2(outputs)
        outputs = self.fc_3(outputs)
        return outputs

4. Huấn luyện mô hình

In [ ]:
# Training function
def train(model, optimizer, criterion, train_dataloader, device, epoch =0, log_interval=50) :
  model.train ()
  total_acc, total_count = 0, 0
  losses = []
  start_time = time.time ()

  for idx , ( inputs , labels ) in enumerate (train_dataloader ):
    inputs = inputs.to (device)
    labels = labels.to (device)

    optimizer.zero_grad()

    predictions = model(inputs)

    # compute loss
    loss = criterion(predictions , labels )
    losses.append(loss.item())

    # backward
    loss.backward()
    torch.nn.utils.clip_grad_norm_( model.parameters (), 0.1)
    optimizer.step ()
    total_acc += ( predictions.argmax (1) == labels ).sum ().item ()
    total_count += labels.size (0)
    if idx % log_interval == 0 and idx > 0:
      elapsed = time.time () - start_time
      print("| epoch {:3d} | {:5d}/{:5d} batches | accuracy {:8.3f}".format(epoch, idx, len(train_dataloader), total_acc / total_count))
      total_acc, total_count = 0, 0
      start_time = time.time()

  epoch_acc = total_acc / total_count
  epoch_loss = sum(losses) / len (losses)
  return epoch_acc , epoch_loss

# Evaluation function
def evaluate(model, criterion , valid_dataloader) :
  model.eval ()
  total_acc, total_count = 0, 0
  losses = []

  with torch.no_grad () :
    for idx, (inputs, labels) in enumerate(valid_dataloader) :
      inputs = inputs.to(device)
      labels = labels.to(device)

      predictions = model(inputs)

      loss = criterion(predictions, labels)
      losses.append (loss.item())

      total_acc += (predictions.argmax (1) == labels).sum().item()
      total_count += labels.size (0)

  epoch_acc = total_acc / total_count
  epoch_loss = sum(losses)/len(losses)
  return epoch_acc , epoch_loss

- Training:

In [ ]:
import time
import os

# Initialize model
num_classes = len( train_data.dataset.classes )

device = torch.device ('cuda' if torch.cuda.is_available () else 'cpu')

lenet_model = LeNetClassifier ( num_classes )
lenet_model.to ( device )

criterion = torch.nn.CrossEntropyLoss ()
optimizer = optim.Adam ( lenet_model.parameters () )

num_epochs = 10
save_model = './model'

train_accs , train_losses = [] , []
eval_accs , eval_losses = [] , []
best_loss_eval = 100

# Create the directory if it doesn't exist
os.makedirs(save_model, exist_ok=True)

for epoch in range (1 , num_epochs +1) :
  epoch_start_time = time.time ()
  # Training
  train_acc , train_loss = train ( lenet_model , optimizer , criterion , train_dataloader , device , epoch )
  train_accs.append ( train_acc )
  train_losses.append ( train_loss )

  # Evaluation
  eval_acc , eval_loss = evaluate ( lenet_model , criterion , valid_dataloader )
  eval_accs.append ( eval_acc )
  eval_losses.append ( eval_loss )

  # Save best model
  if eval_loss < best_loss_eval :
    torch.save ( lenet_model.state_dict () , save_model + '/lenet_model.pt') # Save model

  # Print loss , acc end epoch
  print ("-" * 59)
  print ("| End of epoch {:3d} | Time : {:5.2f}s | Train Accuracy {:8.3f} | Train Loss {:8.3f} | Valid Accuracy {:8.3f} | Valid Loss {:8.3f} ".format (epoch , time.time () - epoch_start_time , train_acc , train_loss , eval_acc , eval_loss))
  print ("-" * 59)

  # Load best model
  lenet_model.load_state_dict ( torch.load ( save_model + '/lenet_model.pt') )
  lenet_model.eval ()

| epoch   1 |    50/  211 batches | accuracy    0.682
| epoch   1 |   100/  211 batches | accuracy    0.870
| epoch   1 |   150/  211 batches | accuracy    0.905
| epoch   1 |   200/  211 batches | accuracy    0.933
-----------------------------------------------------------
| End of epoch   1 | Time : 32.48s | Train Accuracy    0.946 | Train Loss    0.512 | Valid Accuracy    0.949 | Valid Loss    0.182 
-----------------------------------------------------------


<ipython-input-17-338192aaab1a>:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lenet_model . load_state_dict ( torch . load ( save_model + '/lenet_model.pt') )


| epoch   2 |    50/  211 batches | accuracy    0.950
| epoch   2 |   100/  211 batches | accuracy    0.959
| epoch   2 |   150/  211 batches | accuracy    0.962
| epoch   2 |   200/  211 batches | accuracy    0.969
-----------------------------------------------------------
| End of epoch   2 | Time : 30.99s | Train Accuracy    0.967 | Train Loss    0.127 | Valid Accuracy    0.968 | Valid Loss    0.107 
-----------------------------------------------------------
| epoch   3 |    50/  211 batches | accuracy    0.969
| epoch   3 |   100/  211 batches | accuracy    0.971
| epoch   3 |   150/  211 batches | accuracy    0.974
| epoch   3 |   200/  211 batches | accuracy    0.973
-----------------------------------------------------------
| End of epoch   3 | Time : 31.51s | Train Accuracy    0.980 | Train Loss    0.089 | Valid Accuracy    0.974 | Valid Loss    0.095 
-----------------------------------------------------------
| epoch   4 |    50/  211 batches | accuracy    0.975
| epoch   

5. Đánh giá mô hình trên tập test
- Đánh giá độ chính xác trên tập test

In [ ]:
test_data.transform = test_transforms
test_dataloader = data.DataLoader (test_data, batch_size = BATCH_SIZE)
test_acc, test_loss = evaluate (lenet_model , criterion , test_dataloader)
test_acc, test_loss

(0.9859, 0.040506565682881045)

**2.1.2. Bộ dữ liệu Cassava Leaf Disease**



In [ ]:
!wget --no-check-certificate https://storage.googleapis.com/emcassavadata/cassavaleafdata.zip\
-O/content/cassavaleafdata.zip
!unzip /content/cassavaleafdata.zip

# Import libs
import os
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from torchsummary import summary

import matplotlib.pyplot as plt
from PIL import Image

Streaming output truncated to the last 5000 lines.
  inflating: cassavaleafdata/train/cgm/train-cgm-679.jpg  
  inflating: cassavaleafdata/train/cgm/train-cgm-68.jpg  
  inflating: cassavaleafdata/train/cgm/train-cgm-680.jpg  
  inflating: cassavaleafdata/train/cgm/train-cgm-681.jpg  
  inflating: cassavaleafdata/train/cgm/train-cgm-682.jpg  
  inflating: cassavaleafdata/train/cgm/train-cgm-683.jpg  
  inflating: cassavaleafdata/train/cgm/train-cgm-684.jpg  
  inflating: cassavaleafdata/train/cgm/train-cgm-685.jpg  
  inflating: cassavaleafdata/train/cgm/train-cgm-686.jpg  
  inflating: cassavaleafdata/train/cgm/train-cgm-687.jpg  
  inflating: cassavaleafdata/train/cgm/train-cgm-688.jpg  
  inflating: cassavaleafdata/train/cgm/train-cgm-689.jpg  
  inflating: cassavaleafdata/train/cgm/train-cgm-69.jpg  
  inflating: cassavaleafdata/train/cgm/train-cgm-690.jpg  
  inflating: cassavaleafdata/train/cgm/train-cgm-691.jpg  
  inflating: cassavaleafdata/train/cgm/train-cgm-692.jpg  
  infla

In [ ]:
data_paths = {'train': '/content/cassavaleafdata/train', 'valid': '/content/cassavaleafdata/validation', 'test': '/content/cassavaleafdata/test'}

# load image from path
def loader (path) :
  return Image.open (path )

img_size = 150
train_transforms = transforms.Compose ([transforms.Resize ((150 , 150) ) , transforms.ToTensor () ,])

train_data = datasets.ImageFolder ( root = data_paths ['train'] , loader = loader , transform = train_transforms )
valid_data = datasets.ImageFolder ( root = data_paths ['valid'] , transform = train_transforms)
test_data = datasets.ImageFolder (root = data_paths ['test'] , transform = train_transforms )

In [ ]:
class LeNetClassifier ( nn.Module ) :
  def __init__ ( self , num_classes ) :
    super ().__init__ ()
    # Change in_channels to 3 to match the input image channels
    self.conv1 = nn.Conv2d ( in_channels =3 , out_channels =6 , kernel_size =5 , padding ='same')
    self.avgpool1 = nn.AvgPool2d ( kernel_size =2)
    self.conv2 = nn.Conv2d ( in_channels =6 , out_channels =16 , kernel_size =5)
    self.avgpool2 = nn.AvgPool2d ( kernel_size =2)
    self.flatten = nn.Flatten ()
    # Calculate the correct input size for fc_1 dynamically
    # By passing a dummy input through the convolutional layers
    self . fc_1 = nn . Linear (16 * 35 * 35 , 120)
    self . fc_2 = nn . Linear (120 , 84)
    self . fc_3 = nn . Linear (84 , num_classes )
    #dummy_input = torch.randn(1, 3, 150, 150) # Assuming input size 150x150
    #dummy_output = self.avgpool2(F.relu(self.conv2(F.relu(self.avgpool1(self.conv1(dummy_input))))))
    #fc1_input_size = dummy_output.flatten(1).shape[1]
    # Use the calculated size for fc_1
    #self.fc_1 = nn.Linear (fc1_input_size , 120)
    #self.fc_2 = nn.Linear (120 , 84)
    #self.fc_3 = nn.Linear (84 , num_classes )

  def forward(self, inputs):
        outputs = self.conv1(inputs)
        outputs = self.avgpool1(outputs)
        outputs = F.relu(outputs)
        outputs = self.conv2(outputs)
        outputs = self.avgpool2(outputs)
        outputs = F.relu(outputs)
        outputs = self.flatten(outputs)
        outputs = self.fc_1(outputs)
        outputs = self.fc_2(outputs)
        outputs = self.fc_3(outputs)
        return outputs  # Changed 'output' to 'outputs'

In [ ]:
def train(model, optimizer, criterion, train_dataloader, device, epoch=0, log_interval=50):
    model.train()
    total_acc, total_count = 0, 0
    losses = []
    start_time = time.time()

    for idx, (inputs, labels) in enumerate(train_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        predictions = model(inputs)

        # compute loss
        loss = criterion(predictions, labels)
        losses.append(loss.item())

        # backward
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predictions.argmax(1) == labels).sum().item()
        total_count += labels.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "  # Removed extra space in format specifier
                "| accuracy {:8.3f}".format(            # Removed extra space in format specifier
                    epoch, idx, len(train_dataloader), total_acc / total_count if total_count else 0 # Handle zero total_count
                )
            )
            total_acc, total_count = 0, 0
            start_time = time.time()

    # Check if total_count is still 0 after processing all batches
    if total_count == 0:
        print("Warning: No data processed in this epoch. Check your data loader.")
        return 0, float('nan')  # Return 0 for accuracy and NaN for loss

    epoch_acc = total_acc / total_count
    epoch_loss = sum(losses) / len(losses)
    return epoch_acc, epoch_loss

In [ ]:
def evaluate ( model , criterion , valid_dataloader ) :
  model . eval ()
  total_acc , total_count = 0 , 0
  losses = []

  with torch . no_grad () :
    for idx , ( inputs , labels ) in enumerate ( valid_dataloader ) :
      inputs = inputs . to ( device )
      labels = labels . to ( device )

      predictions = model ( inputs )

      loss = criterion ( predictions , labels )
      losses . append ( loss . item () )

      total_acc += ( predictions . argmax (1) == labels ) . sum () . item ()
      total_count += labels . size (0)

  epoch_acc = total_acc / total_count
  epoch_loss = sum( losses ) / len ( losses )
  return epoch_acc , epoch_loss

In [ ]:
!rm -rf /content/model

In [ ]:
import time
import os
import torch.utils.data as data # Make sure to import data

BATCH_SIZE = 256  # Adjust the batch size as needed
train_dataloader = data.DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE)
valid_dataloader = data.DataLoader(valid_data, batch_size=BATCH_SIZE) # It's good practice to define valid_dataloader here as well
test_dataloader = data.DataLoader(test_data, batch_size=BATCH_SIZE) # It's good practice to define test_dataloader here as well


# Initialize model
num_classes = len( train_data.classes )

device = torch.device ('cuda' if torch.cuda.is_available () else 'cpu')

lenet_model = LeNetClassifier ( num_classes )
lenet_model.to ( device )

criterion = torch.nn.CrossEntropyLoss ()
learning_rate = 2e-4
optimizer = optim.Adam ( lenet_model.parameters () , learning_rate )

num_epochs = 10
save_model = '/content/model'

# Create the directory if it doesn't exist
os.makedirs(save_model, exist_ok=True)

train_accs , train_losses = [] , []
eval_accs , eval_losses = [] , []
best_loss_eval = 100

for epoch in range (1 , num_epochs +1) :
  epoch_start_time = time.time ()
  # Training
  train_acc , train_loss = train ( lenet_model , optimizer , criterion , train_dataloader, device , epoch , log_interval =10)
  train_accs.append ( train_acc )
  train_losses.append ( train_loss )

  # Evaluation
  eval_acc , eval_loss = evaluate ( lenet_model , criterion , valid_dataloader )
  eval_accs.append ( eval_acc )
  eval_losses.append ( eval_loss )

  # Save best model
  if eval_loss < best_loss_eval :
    torch.save ( lenet_model.state_dict () , save_model + '/lenet_model.pt')

  # Print loss , acc end epoch
  print ("-" * 59)
  print(
    "| End of epoch {:3d} | Time: {:5.2f}s | Train Accuracy: {:8.3f} | Train Loss: {:8.3f} "
    "| Valid Accuracy: {:8.3f} | Valid Loss: {:8.3f} ".format(
        epoch, time.time() - epoch_start_time, train_acc, train_loss, eval_acc, eval_loss
    )
  )
  print ("-" * 59)

  # Load best model
  lenet_model.load_state_dict ( torch.load (save_model + '/lenet_model.pt') )
  lenet_model.eval ()

| epoch   1 |    10/   23 batches | accuracy    0.434
| epoch   1 |    20/   23 batches | accuracy    0.470
-----------------------------------------------------------
| End of epoch   1 | Time: 47.15s | Train Accuracy:    0.471 | Train Loss:    1.367 | Valid Accuracy:    0.470 | Valid Loss:    1.576 
-----------------------------------------------------------


<ipython-input-23-cfc8fd05a6cd>:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lenet_model.load_state_dict ( torch.load (save_model + '/lenet_model.pt') )


| epoch   2 |    10/   23 batches | accuracy    0.465
| epoch   2 |    20/   23 batches | accuracy    0.475
-----------------------------------------------------------
| End of epoch   2 | Time: 47.12s | Train Accuracy:    0.507 | Train Loss:    1.366 | Valid Accuracy:    0.470 | Valid Loss:    1.446 
-----------------------------------------------------------
| epoch   3 |    10/   23 batches | accuracy    0.474
| epoch   3 |    20/   23 batches | accuracy    0.468
-----------------------------------------------------------
| End of epoch   3 | Time: 48.07s | Train Accuracy:    0.443 | Train Loss:    1.328 | Valid Accuracy:    0.471 | Valid Loss:    1.431 
-----------------------------------------------------------
| epoch   4 |    10/   23 batches | accuracy    0.479
| epoch   4 |    20/   23 batches | accuracy    0.480
-----------------------------------------------------------
| End of epoch   4 | Time: 46.93s | Train Accuracy:    0.457 | Train Loss:    1.316 | Valid Accuracy:    0

In [ ]:
test_acc , test_loss = evaluate ( lenet_model , criterion , test_dataloader )
test_acc , test_loss

(0.5246684350132625, 1.347385734319687)

## **2.2. Phân loại văn bản**

In [ ]:
!git clone https://github.com/congnghia0609/ntc-scv.git
!unzip ./ntc-scv/data/data_test.zip -d ./data
!unzip ./ntc-scv/data/data_train.zip -d ./data
!rm -rf ./ntc-scv

# Import thư viện cần thiết
import os
import pandas as pd

# Hàm load dữ liệu từ đường dẫn vào DataFrame
def load_data_from_path(folder_path):
    examples = []
    for label in os.listdir(folder_path):  # Lặp qua các thư mục con (label)
        full_path = os.path.join(folder_path, label)
        for file_name in os.listdir(full_path):  # Lặp qua các file trong từng thư mục
            file_path = os.path.join(full_path, file_name)
            with open(file_path, "r", encoding="utf-8") as f:
                lines = f.readlines()
                sentence = "".join(lines).strip()  # Ghép nội dung file thành 1 câu
                data = {
                    "sentence": sentence,
                    "label": 0 if label == "neg" else 1  # Gán nhãn 0 cho "neg" và 1 cho "pos"
                }
                examples.append(data)  # Thêm vào danh sách
    return pd.DataFrame(examples)  # Trả về DataFrame từ danh sách

# Định nghĩa các đường dẫn dữ liệu
folder_paths = {
    "train": "./data/data_train/train",
    "valid": "./data/data_train/valid",
    "test": "./data/data_test/test"
}

# Load dữ liệu vào các DataFrame
train_df = load_data_from_path(folder_paths["train"])
valid_df = load_data_from_path(folder_paths["valid"])
test_df = load_data_from_path(folder_paths["test"])

Cloning into 'ntc-scv'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 39 (delta 0), reused 4 (delta 0), pack-reused 35 (from 1)
Receiving objects: 100% (39/39), 186.94 MiB | 13.98 MiB/s, done.
Resolving deltas: 100% (9/9), done.
Archive:  ./ntc-scv/data/data_test.zip
replace ./data/data_test/test/neg/10.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# Cài đặt thư viện cần thiết
!pip install langid

# Import thư viện
from langid.langid import LanguageIdentifier, model

# Hàm nhận diện ngôn ngữ tiếng Việt trong DataFrame
def identify_vn(df):
    # Tạo đối tượng LanguageIdentifier
    identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)

    # Ngưỡng xác suất cho ngôn ngữ tiếng Việt
    THRESHOLD = 0.9

    # Tập hợp các chỉ mục không phải tiếng Việt
    not_vi_idx = set()

    # Duyệt qua từng dòng trong DataFrame
    for idx, row in df.iterrows():
        # Phân loại ngôn ngữ của câu
        score = identifier.classify(row["sentence"])

        # Kiểm tra điều kiện không phải tiếng Việt
        if score[0] != "vi" or (score[0] == "vi" and score[1] <= THRESHOLD):
            not_vi_idx.add(idx)

    # Tạo DataFrame tiếng Việt và không phải tiếng Việt
    vi_df = df[~df.index.isin(not_vi_idx)]
    not_vi_df = df[df.index.isin(not_vi_idx)]

    return vi_df, not_vi_df

# Phân loại ngôn ngữ trong tập dữ liệu train
train_df_vi, train_df_other = identify_vn(train_df)

NameError: name 'train_df' is not defined

In [ ]:
# Import các thư viện cần thiết
import re
import string

# Hàm tiền xử lý văn bản
def preprocess_text(text):
    # Loại bỏ URL
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    text = url_pattern.sub(r' ', text)

    # Loại bỏ thẻ HTML
    html_pattern = re.compile(r'<.*?>')
    text = html_pattern.sub(r' ', text)

    # Loại bỏ ký tự đặc biệt và chữ số
    replace_chars = string.punctuation + string.digits
    for char in replace_chars:
        text = text.replace(char, " ")

    # Loại bỏ emoji
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U0001F300-\U0001F5FF"  # Symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # Transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # Flags
        u"\U00002702-\U000027B0"  # Miscellaneous symbols
        u"\U000024C2-\U0001F251"  # Enclosed characters
        u"\U0001F926-\U0001F937"  # Supplemental symbols
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # Regional indicators
        u"\U0001F600-\U0001F64F"  # Additional emoticons
        u"\u200d"                 # Zero-width joiner
        u"\u2640-\u2642"          # Gender symbols
        "]+", flags=re.UNICODE
    )
    text = emoji_pattern.sub(r' ', text)

    # Chuẩn hóa khoảng trắng và chuyển thành chữ thường
    text = " ".join(text.split())
    return text.lower()

# Áp dụng hàm tiền xử lý lên các DataFrame
train_df_vi['preprocess_sentence'] = [
    preprocess_text(row['sentence']) for _, row in train_df_vi.iterrows()
]

valid_df['preprocess_sentence'] = [
    preprocess_text(row['sentence']) for _, row in valid_df.iterrows()
]

test_df['preprocess_sentence'] = [
    preprocess_text(row['sentence']) for _, row in test_df.iterrows()
]

<ipython-input-27-b686b891db16>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_vi['preprocess_sentence'] = [


In [ ]:
! pip install -q torchtext ==0.16.0

# Import các thư viện cần thiết
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset

# Tokenizer theo từ
tokenizer = get_tokenizer("basic_english")

# Hàm tạo bộ token từ các câu
def yield_tokens(sentences, tokenizer):
    for sentence in sentences:
        yield tokenizer(sentence)

# Xây dựng từ điển (vocabulary)
vocab_size = 10000
vocabulary = build_vocab_from_iterator(
    yield_tokens(train_df_vi['preprocess_sentence'], tokenizer),
    max_tokens=vocab_size,
    specials=["<pad>", "<unk>"]
)
vocabulary.set_default_index(vocabulary["<unk>"])

# Hàm chuẩn bị dữ liệu để chuyển sang dataset của TorchText
def prepare_dataset(df):
    for _, row in df.iterrows():
        sentence = row['preprocess_sentence']
        encoded_sentence = vocabulary(tokenizer(sentence))
        label = row['label']
        yield encoded_sentence, label

# Chuyển đổi dữ liệu thành dataset của TorchText
train_dataset = to_map_style_dataset(prepare_dataset(train_df_vi))
valid_dataset = to_map_style_dataset(prepare_dataset(valid_df))
test_dataset = to_map_style_dataset(prepare_dataset(test_df))

ERROR: Invalid requirement: '==0.16.0': Expected package name at the start of dependency specifier
    ==0.16.0
    ^



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/ke

NameError: name 'train_df_vi' is not defined

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# Hàm chuẩn bị batch dữ liệu (collate function)
def collate_batch(batch):
    # Khởi tạo danh sách lưu sentences và labels
    encoded_sentences, labels = [], []

    for encoded_sentence, label in batch:
        # Chuyển encoded_sentence và label thành tensor
        encoded_sentence = torch.tensor(encoded_sentence, dtype=torch.int64)
        labels.append(label)
        encoded_sentences.append(encoded_sentence)

    # Chuyển labels thành tensor
    labels = torch.tensor(labels, dtype=torch.int64)

    # Padding các sentences để có cùng độ dài
    encoded_sentences = pad_sequence(
        encoded_sentences,
        batch_first=True,
        padding_value=vocabulary["<pad>"]
    )

    return encoded_sentences, labels

# Định nghĩa kích thước batch
batch_size = 128

# Tạo DataLoader cho tập train
train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,  # Shuffle để đảm bảo dữ liệu đa dạng trong từng epoch
    collate_fn=collate_batch
)

# Tạo DataLoader cho tập validation
valid_dataloader = DataLoader(
    valid_dataset,
    batch_size=batch_size,
    shuffle=False,  # Không shuffle để giữ thứ tự dữ liệu
    collate_fn=collate_batch
)

# Tạo DataLoader cho tập test
test_dataloader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,  # Không shuffle
    collate_fn=collate_batch
)

NameError: name 'train_dataset' is not defined

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# Định nghĩa mô hình TextCNN
class TextCNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, kernel_sizes, num_filters, num_classes):
        super(TextCNN, self).__init__()

        # Lưu các tham số
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.kernel_sizes = kernel_sizes
        self.num_filters = num_filters
        self.num_classes = num_classes

        # Lớp embedding để ánh xạ từ thành vector
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)

        # Tạo các lớp Convolutional với nhiều kích thước kernel
        self.conv = nn.ModuleList([
            nn.Conv1d(
                in_channels=embedding_dim,
                out_channels=num_filters,
                kernel_size=k,
                stride=1
            )
            for k in kernel_sizes
        ])

        # Lớp fully connected để phân loại
        self.fc = nn.Linear(len(kernel_sizes) * num_filters, num_classes)

    def forward(self, x):
        # Đầu vào x có kích thước: (batch_size, sequence_length)
        batch_size, sequence_length = x.shape

        # Chuyển đầu vào qua lớp embedding: (batch_size, sequence_length, embedding_dim)
        x = self.embedding(x)

        # Chuyển đổi để phù hợp với Conv1d: (batch_size, embedding_dim, sequence_length)
        x = x.transpose(1, 2)

        # Áp dụng các lớp Convolutional + ReLU + Max Pooling
        x = [F.relu(conv(x)) for conv in self.conv]  # (batch_size, num_filters, new_length)
        x = [F.max_pool1d(c, kernel_size=c.size(2)).squeeze(2) for c in x]  # (batch_size, num_filters)

        # Nối các output từ các lớp convolution lại
        x = torch.cat(x, dim=1)  # (batch_size, len(kernel_sizes) * num_filters)

        # Đưa qua lớp fully connected
        x = self.fc(x)  # (batch_size, num_classes)

        return x

In [ ]:
import time
import torch


# Hàm huấn luyện (training)
def train(model, optimizer, criterion, train_dataloader, device, epoch=0, log_interval=50):
    model.train()  # Chuyển model sang chế độ training
    total_acc, total_count = 0, 0  # Biến đếm tổng số chính xác và tổng số mẫu
    losses = []  # Danh sách lưu các giá trị loss
    start_time = time.time()  # Ghi lại thời gian bắt đầu

    for idx, (inputs, labels) in enumerate(train_dataloader):
        # Chuyển dữ liệu và nhãn sang thiết bị (device)
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Reset gradient
        optimizer.zero_grad()

        # Dự đoán
        predictions = model(inputs)

        # Tính toán loss
        loss = criterion(predictions, labels)
        losses.append(loss.item())

        # Lan truyền ngược (backpropagation)
        loss.backward()
        optimizer.step()

        # Cập nhật độ chính xác và số lượng
        total_acc += (predictions.argmax(1) == labels).sum().item()
        total_count += labels.size(0)

        # In log mỗi log_interval batch
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(f"| epoch {epoch:3d} | {idx:5d}/{len(train_dataloader):5d} batches "
                  f"| accuracy {total_acc / total_count:.3f} | time elapsed {elapsed:.2f}s")

            total_acc, total_count = 0, 0  # Reset cho log tiếp theo
            start_time = time.time()  # Ghi lại thời gian bắt đầu log tiếp theo

    # Tính toán độ chính xác và loss của epoch
    epoch_acc = total_acc / total_count
    epoch_loss = sum(losses) / len(losses)

    return epoch_acc, epoch_loss


# Hàm đánh giá (evaluation)
def evaluate(model, criterion, valid_dataloader, device):
    model.eval()  # Chuyển model sang chế độ evaluation
    total_acc, total_count = 0, 0  # Biến đếm tổng số chính xác và tổng số mẫu
    losses = []  # Danh sách lưu các giá trị loss

    with torch.no_grad():  # Tắt tính toán gradient
        for idx, (inputs, labels) in enumerate(valid_dataloader):
            # Chuyển dữ liệu và nhãn sang thiết bị (device)
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Dự đoán
            predictions = model(inputs)

            # Tính toán loss
            loss = criterion(predictions, labels)
            losses.append(loss.item())

            # Cập nhật độ chính xác và số lượng
            total_acc += (predictions.argmax(1) == labels).sum().item()
            total_count += labels.size(0)

    # Tính toán độ chính xác và loss của epoch
    epoch_acc = total_acc / total_count
    epoch_loss = sum(losses) / len(losses)

    return epoch_acc, epoch_loss

In [ ]:
!pip install -q torchtext==0.16.0
import torch
import time

# Thiết lập thông số
num_classes = 2
vocab_size = len(vocabulary)
embedding_dim = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Khởi tạo mô hình TextCNN
model = TextCNN(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    kernel_sizes=[3, 4, 5],
    num_filters=100,
    num_classes=num_classes,
)
model.to(device)  # Đưa mô hình lên thiết bị (CPU/GPU)

# Định nghĩa hàm mất mát và bộ tối ưu
criterion = torch.nn.CrossEntropyLoss()

# Wrap optimizer creation within the context manager
with torch.no_grad(): # Stop tracing and compilation for Adam init and param
    optimizer = torch.optim.Adam(model.parameters())

# Số epochs
num_epochs = 10
save_model_path = "./model"

# Biến lưu kết quả
train_accuracies, train_losses = [], []
eval_accuracies, eval_losses = [], []
best_loss_eval = float("inf")  # Đặt giá trị loss tốt nhất ban đầu là vô cùng lớn

# Vòng lặp huấn luyện và đánh giá
for epoch in range(1, num_epochs + 1):
    # Ghi lại thời gian bắt đầu epoch
    epoch_start_time = time.time()

    # Huấn luyện
    train_acc, train_loss = train(
        model, optimizer, criterion, train_dataloader, device, epoch
    )
    train_accuracies.append(train_acc)
    train_losses.append(train_loss)

    # Đánh giá
    eval_acc, eval_loss = evaluate(
        model, criterion, valid_dataloader, device
    )
    eval_accuracies.append(eval_acc)
    eval_losses.append(eval_loss)

    # Lưu mô hình tốt nhất
    if eval_loss < best_loss_eval:
        best_loss_eval = eval_loss
        torch.save(model.state_dict(), f"{save_model_path}/text_cnn_model.pt")

    # In thông tin sau mỗi epoch
    print("-" * 59)
    print(
        f"End of epoch {epoch:3d} | Time: {time.time() - epoch_start_time:5.2f}s "
        f"| Train Accuracy: {train_acc:8.3f} | Train Loss: {train_loss:8.3f} "
        f"| Valid Accuracy: {eval_acc:8.3f} | Valid Loss: {eval_loss:8.3f}"
    )
    print("-" * 59)

# Load mô hình tốt nhất
model.load_state_dict(torch.load(f"{save_model_path}/text_cnn_model.pt"))
model.eval()  # Chuyển mô hình sang chế độ đánh giá

ImportError: cannot import name 'is_fbcode' from partially initialized module 'torch._dynamo.config' (most likely due to a circular import) (/usr/local/lib/python3.10/dist-packages/torch/_dynamo/config.py)

In [ ]:
# Chuẩn bị tập dữ liệu test
test_dataset = prepare_dataset(test_df)  # Chuẩn bị dữ liệu từ DataFrame
test_dataset = to_map_style_dataset(test_dataset)  # Chuyển đổi sang dạng MapStyleDataset

# Tạo DataLoader cho tập test
test_dataloader = DataLoader(
    test_dataset,
    batch_size=batch_size,  # Sử dụng kích thước batch đã định nghĩa trước
    shuffle=False,  # Không shuffle dữ liệu test
    collate_fn=collate_batch  # Hàm gộp batch
)

# Đánh giá mô hình trên tập test
test_acc, test_loss = evaluate(model, criterion, test_dataloader, device)

# Hiển thị kết quả đánh giá
print(f"Test Accuracy: {test_acc:.3f}, Test Loss: {test_loss:.3f}")

Test Accuracy: 0.500, Test Loss: 1.433
